In [1]:
import os
import json

import numpy as np

from pyne import data
from pyne import nucname
import transmutagen

/home/scopatz/miniconda/lib/python3.5/importlib/_bootstrap.py:222: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)


In [2]:
json_file = os.path.join(os.path.dirname(transmutagen.__file__), 'data/gensolve.json')
with open(json_file) as f:
    gsdata = json.load(f)

In [3]:
D = np.zeros(len(gsdata['fromto']), dtype='f8')
# note that there is a bug in transmutagen! tofrom is actually fromto
for i, (f, t) in enumerate(gsdata['fromto']): 
    decay_const = data.decay_const(f)
    if decay_const <= 0.0 or np.isnan(decay_const):
        continue
    elif f == t:
        D[i] = -decay_const
    else:
        br = data.branch_ratio(f, t)
        D[i] = decay_const * br

In [4]:
D[:20]

array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   1.78283350e-09,
        -1.78283350e-09,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,  -3.43723479e+22,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,  -8.48699947e+22,
         8.59237859e-01,  -8.59237859e-01])

In [5]:
gsdata['fromto'][8]

['H4', 'H4']

In [6]:
data.branch_ratio('U235', 'Tc99')

4.29982e-12

In [7]:
tr_nucs = {x for x in gsdata['nucs']}
tr_fromto = {tuple(x) for x in gsdata['fromto']}

In [8]:
data.atomic_mass('U235')
pyne_nucs = set()
for i in data.atomic_mass_map.keys():
    if not nucname.isnuclide(i):
        continue
    try:
        n = nucname.name(i)
    except:
        continue
    pyne_nucs.add(n)

In [9]:
pyne_nucs - tr_nucs

set()

# Good, since this set difference is empty, transmutagen already has all of the nuclides that pyne tracks.

# Now to check that PyNE doesn't add any branches that transmutagen is unaware of

In [10]:
pyne_fromto = set()
for nuc in tr_nucs:
    f = nucname.name(nuc)
    kids = data.decay_children(nuc)
    fts = {(f, nucname.name(t)) for t in kids}
    pyne_fromto.update(fts)

In [11]:
len(pyne_fromto - tr_fromto)

2997

In [12]:
bad = set()
for f, t in pyne_fromto:
    decay_const = data.decay_const(f)
    br = data.branch_ratio(f, t)
    if decay_const <= 0.0 or np.isnan(decay_const):
        print('The decay const of ' + f + ' is invalid: ' + str(decay_const))
    elif br <= 0.0 or np.isnan(br):
        print('The branch ratio of ' + f + ', ' + t + ' is invalid: ' + str(br))
    bad.add((f, t))

The branch ratio of U238, Ni75 is invalid: 0.0
The branch ratio of Pu236, Pu234 is invalid: 0.0
The branch ratio of Pu239, Sr86 is invalid: 0.0
The branch ratio of Pu239, Ba143 is invalid: 0.0
The branch ratio of U238, Mo102 is invalid: 0.0
The branch ratio of U238, Gd154 is invalid: 0.0
The branch ratio of Cf249, Pr141 is invalid: 0.0
The branch ratio of U235, Cd126 is invalid: 0.0
The branch ratio of Cm245, Tc99 is invalid: 0.0
The branch ratio of Th232, Sn125 is invalid: 0.0
The branch ratio of U235, Se79M is invalid: 0.0
The branch ratio of Cf249, Y104 is invalid: 0.0
The branch ratio of Pu242, Eu151 is invalid: 0.0
The branch ratio of U238, Tb159 is invalid: 0.0
The branch ratio of Cm245, Xe135M is invalid: 0.0
The branch ratio of Cf249, Sn118 is invalid: 0.0
The branch ratio of Pu239, Ag114 is invalid: 0.0
The branch ratio of Cf249, Te131 is invalid: 0.0
The branch ratio of Cm245, Mo97 is invalid: 0.0
The branch ratio of Cm245, Pd111 is invalid: 0.0
The branch ratio of Pu239, Sn1

The branch ratio of Cm245, Xe137 is invalid: 0.0
The branch ratio of U238, Ru112 is invalid: 0.0
The branch ratio of Pu239, Te128 is invalid: 0.0
The branch ratio of Th232, Rh119 is invalid: 0.0
The branch ratio of U238, Ba143 is invalid: 0.0
The branch ratio of U235, Sb134 is invalid: 0.0
The branch ratio of Th232, Xe143 is invalid: 0.0
The branch ratio of Cf249, Rh107 is invalid: 0.0
The branch ratio of U235, Sn116 is invalid: 0.0
The branch ratio of Cf249, Cd120 is invalid: 0.0
The branch ratio of Cf249, Ga75 is invalid: 0.0
The branch ratio of Cf249, Ho166M is invalid: 0.0
The branch ratio of U235, Sb123 is invalid: 0.0
The branch ratio of Cm245, Cs146 is invalid: 0.0
The branch ratio of Cm245, Mo105 is invalid: 0.0
The branch ratio of U235, Tc111 is invalid: 0.0
The branch ratio of U238, Tc100 is invalid: 0.0
The branch ratio of U238, Pd117 is invalid: 0.0
The branch ratio of Pu239, Rh105M is invalid: 0.0
The branch ratio of Cm245, La139 is invalid: 0.0
The branch ratio of U235, I

The branch ratio of Th232, Pm152 is invalid: 0.0
The branch ratio of Cm245, Tb162 is invalid: 0.0
The branch ratio of Cf249, Sn127M is invalid: 0.0
The branch ratio of U238, Pd123 is invalid: 0.0
The branch ratio of Cf249, Rb93 is invalid: 0.0
The branch ratio of Th232, Rb93 is invalid: 0.0
The branch ratio of Pu239, Ru105 is invalid: 0.0
The branch ratio of Cm245, Pr152 is invalid: 0.0
The branch ratio of Th232, Se79M is invalid: 0.0
The branch ratio of Cf249, Ce152 is invalid: 0.0
The branch ratio of Cm245, Ru99 is invalid: 0.0
The branch ratio of Cf249, Se79M is invalid: 0.0
The branch ratio of Pu239, Nb104 is invalid: 0.0
The branch ratio of Th232, Ba134 is invalid: 0.0
The branch ratio of U235, Gd161 is invalid: 0.0
The branch ratio of Th232, Ce151 is invalid: 0.0
The branch ratio of U232, U233 is invalid: 0.0
The branch ratio of U235, Ge72 is invalid: 0.0
The branch ratio of Pu239, Eu160 is invalid: 0.0
The branch ratio of Pu239, Cd115M is invalid: 0.0
The branch ratio of Cf249, 

In [13]:
pyne_fromto -= bad
len(pyne_fromto - tr_fromto)

0

# So the only decay modes PyNE has that transmutagen doesn't are invalid or trivial (br = 0 or lambda = 0). This is great news!